In [1]:
#import ml packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import string

#import nltk packages
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
#import dataset
train_df = pd.read_csv('/home/owner/Desktop/ML/project/AskReddit Dataset/train.csv')
test_df = pd.read_csv('/home/owner/Desktop/ML/project/AskReddit Dataset/test.csv')
train_df.head()

,qid,question_text,target
0,a3dee568776c08512c89,What is the role of Lua in Civ4?,0
1,bdb84f519e7b46e7b7bb,What are important chapters in Kannada for 10 ...,0
2,29c88db470e2eb5c97ad,Do musicians get royalties from YouTube?,0
3,3387d99bf2c3227ae8f1,What is the difference between Scaling Social ...,0
4,e79fa5038f765d0f2e7e,Why do elevators go super slow right before th...,0


In [3]:
# choose elements from df where target = 1
pd.set_option('display.max_colwidth', 100)
df_1 = train_df[train_df['target'] == 1]
df_1["question_text"]

16                                                     What stupid things do Indians do when in your country?
31                             Can I sue my parents for giving birth to me when I did not want them to do so?
32                          What are your views about sexual relationship between a widow mother and her son?
33        You became an atheist, and after 2 years you fall and break your back. You are left paralyzed fr...
90                                    Why aren't we protesting for government control instead of gun control?
                                                         ...                                                 
652967              What is a liberal's understanding of the difference between pollution and climate change?
653021    Do unattractive or average-looking men ever get a girlfriend who actually loves them or do they ...
653029                                                                   How can I grab my aunties boobs! :p?
653034    

In [4]:
# see value count order of target
train_df['target'].value_counts()

0    612656
1     40405
Name: target, dtype: int64

In [5]:
# percentage of troll questions in the dataset
train_df['target'].value_counts(normalize=True)

0    0.93813
1    0.06187
Name: target, dtype: float64

In [11]:
# create a preprocessing class to perform preprocessing
class Preprocessor:
    
    def __init__(self,df) -> None:
        self.df = df
        # nltk.download('stopwords')

    def removePunctuation(self):
        self.df['question_text'] = self.df['question_text'].apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
        return self.df


    def removeStopWords(self):
        stop = stopwords.words('english')
        self.df['question_text'] = self.df['question_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
        return self.df

    def removeNumbers(self):
        self.df['question_text'] = self.df['question_text'].apply(lambda x: ' '.join([word for word in x.split() if not word.isnumeric()]))
        return self.df

    def removeShortWords(self):
        self.df['question_text'] = self.df['question_text'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
        return self.df

    def removeLowOccuranceWords(self):
        freq = pd.Series(' '.join(self.df['question_text']).split()).value_counts()
        low_freq = list(freq.loc[freq<5].index)
        self.df['question_text'] = self.df['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in low_freq))

    def removeUnwantedCols(self,col):
        self.df = self.df.drop(col,axis=1)
        return self.df
    
    def preprocess(self):
        self.removePunctuation()
        self.removeStopWords()
        self.removeNumbers()
        self.removeShortWords()
        self.removeUnwantedCols(['qid'])
        return self.df

In [12]:
preprocessor = Preprocessor(train_df)
preprocessed_df = preprocessor.preprocess()
preprocessed_df.head()

,question_text,target
0,What role Lua Civ4,0
1,What important chapters Kannada ICSE,0
2,musicians get royalties YouTube,0
3,What difference Scaling Social Enterprises Social Franchising,0
4,Why elevators super slow right doors open,0


In [14]:
# do somewhat more preprocessing
# tokenize words using nltk
nltk.download('punkt')
preprocessed_df['question_text'] = preprocessed_df['question_text'].apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to /home/owner/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
preprocessed_df.head()

,question_text,target
0,"[What, role, Lua, Civ4]",0
1,"[What, important, chapters, Kannada, ICSE]",0
2,"[musicians, get, royalties, YouTube]",0
3,"[What, difference, Scaling, Social, Enterprises, Social, Franchising]",0
4,"[Why, elevators, super, slow, right, doors, open]",0
